# Viewing angles

Visualizing the illumination and viewing angles. Explaining how ViSiProg groups are related?

In [8]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import plotly.graph_objs as go
from visiprog.data import read_VSP_label, read_img_list, get_viewing_condition

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def polar_to_euclidean(theta, phi):
    '''
    theta is polar angle (angle from z axis)
    phi is azimuth angle (angle from x axis to projection)
    
    '''
    X = np.sin(theta) * np.cos(phi)
    Y = np.sin(theta) * np.sin(phi)
    Z = np.cos(theta)
    return (X, Y, Z)


def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    def unit_vector(vector):
        """ Returns the unit vector of the vector.  """
        return vector / np.linalg.norm(vector)

    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

In [3]:
viewing = np.genfromtxt('visiprog/data/viewing.csv',delimiter=',')
print(viewing.shape)

(205, 5)


In [4]:
trace_illum_x = []
trace_illum_y = []
trace_illum_z = []

trace_viewing_x = []
trace_viewing_y = []
trace_viewing_z = []

for i in range(viewing.shape[0]):
    illum_x, illum_y, illum_z = polar_to_euclidean(viewing[i,3],viewing[i,4])
    view_x, view_y, view_z = polar_to_euclidean(viewing[i,1],viewing[i,2])
    
    line_x = [illum_x, 0, view_x]
    line_y = [illum_y, 0, view_y]
    line_z = [illum_z, 0, view_z]
    
    trace_illum_x.append(illum_x)
    trace_illum_y.append(illum_y)
    trace_illum_z.append(illum_z)
    
    trace_viewing_x.append(view_x)
    trace_viewing_y.append(view_y)
    trace_viewing_z.append(view_z)

    
illum_scatter = go.Scatter3d(
    x=trace_illum_x,
    y=trace_illum_y,
    z=trace_illum_z,
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)

In [5]:
layout = go.Layout(
    title='Illumination angles',
    autosize=True,
    width=900,
    height=800,
    margin=dict(
        l=65,
        r=50,
        b=65,
        t=90
    )
)
fig = go.Figure(data=[illum_scatter], layout=layout)
iplot(fig, filename='illumination')

In [6]:
groups, N = read_VSP_label()

list_img = read_img_list()

traces = []

for i in groups[300]: 
    viewing_index = get_viewing_condition(list_img[i])
    
    illum_x, illum_y, illum_z = polar_to_euclidean(viewing[viewing_index,3],viewing[viewing_index,4])
    view_x, view_y, view_z = polar_to_euclidean(viewing[viewing_index,1],viewing[viewing_index,2])
    
    line_x = [illum_x, 0, view_x]
    line_y = [illum_y, 0, view_y]
    line_z = [illum_z, 0, view_z]
    
    # plotting line
    line_scatter = go.Scatter3d(
        x=line_x,
        y=line_y,
        z=line_z,
        name="pair_{}".format(i)
    )
    
    traces.append(line_scatter)
    

    
illum_scatter = go.Scatter3d(
    x=trace_illum_x,
    y=trace_illum_y,
    z=trace_illum_z,
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    title='Illumination angles',
    autosize=True,
    width=900,
    height=800,
    margin=dict(
        l=65,
        r=50,
        b=65,
        t=90
    )
)
fig = go.Figure(data=traces, layout=layout)
iplot(fig, filename='elevations-3d-surface')

# Formal analysis using graph and minimum spanning tree

In [34]:
groups, N = read_VSP_label()
list_img = read_img_list()


mst_distances = []

for group in groups:
    
    print("=================================================")
    
    print(group)
    
    # building the weighted undirected graph
    G = nx.Graph()
    G.add_nodes_from(range(len(group)))

    for i in range(len(group)):
        index_i = get_viewing_condition(list_img[i])
        illum_x_1, illum_y_1, illum_z_1 = polar_to_euclidean(viewing[index_i,3],viewing[index_i,4])
        view_x_1, view_y_1, view_z_1 = polar_to_euclidean(viewing[index_i,1],viewing[index_i,2])

        for j in range(i + 1, len(group)):
            index_j = get_viewing_condition(list_img[j])
            illum_x_2, illum_y_2, illum_z_2 = polar_to_euclidean(viewing[index_j,3],viewing[index_j,4])
            view_x_2, view_y_2, view_z_2 = polar_to_euclidean(viewing[index_j,1],viewing[index_j,2])
            
            weight_illum = angle_between((illum_x_1, illum_y_1, illum_z_1),(illum_x_2, illum_y_2, illum_z_2))
            weight_view = angle_between((view_x_1, view_y_1, view_z_1),(view_x_2, view_y_2, view_z_2))
        
            G.add_edge(i, j, weight=weight_illum)
            G.add_edge(i, j, weight=weight_view)
            
#     print(G.edges(data=True))
#     print("\n\nSorting the MST\n\n")
    
    # minimum spanning tree
    mst=nx.minimum_spanning_edges(G,data=True)
    
    distances = []
    for e in mst:
        distances.append(e[2]['weight'])
        
    mst_distances.append(distances)

[1789, 1787, 1775, 1778, 1782, 1828, 1830, 1781, 1811]
[4357, 4355, 4374, 4345, 4347, 4376, 4354, 4349, 4339]
[2635, 981, 1007, 980, 1042, 2609, 2655, 978, 1004]
[4283, 4305, 4278, 4281, 4250, 4248, 4251, 4249, 4302]
[3599, 3630, 3597, 3633, 3598, 3676, 3629, 3655, 3634]
[1606, 1629, 1626, 1630, 1648, 1595, 1649, 1628, 1662]
[1973, 1970, 1968, 1961, 1972, 1964, 1963, 1969, 1971]
[4898, 4892, 4894, 4923, 4895, 4905, 4907, 4930, 4900]
[4801, 4870, 4854, 4852, 4850, 4828, 4831, 4794, 4826]
[4564, 4532, 4541, 4527, 4535, 4524, 4554, 4540, 4557]
[2351, 2349, 2372, 2347, 2374, 2317, 2373, 2371, 2316]
[1121, 1070, 1099, 1072, 1069, 1119, 1097, 1059, 1098]
[3066, 3112, 3098, 3129, 3061, 3065, 3094, 3068, 3113]
[2270, 2240, 2269, 2242, 2247, 2263, 2236, 2285, 2266]
[4076, 4069, 4077, 4070, 4068, 4098, 4067, 4100, 4122]
[1537, 1511, 1499, 1507, 1497, 1570, 1509, 1508, 1552]
[819, 817, 842, 789, 790, 820, 822, 786, 818]
[3339, 3345, 3341, 3335, 3363, 3372, 3369, 3340, 3368]
[1314, 1295, 1280, 131

In [35]:
mst_out = np.asarray(mst_distances)
np.savetxt('mst_distances.csv', mst_out, fmt='%.4f', delimiter=',')

In [28]:
print(mst_out)

[[ 0.06266034  0.11125446  0.12600595  0.16634496  0.18049313  0.20986609
   0.23138332  0.26586011]
 [ 0.06266034  0.11125446  0.12600595  0.16634496  0.18049313  0.20986609
   0.23138332  0.26586011]
 [ 0.06266034  0.11125446  0.12600595  0.16634496  0.18049313  0.20986609
   0.23138332  0.26586011]
 [ 0.06266034  0.11125446  0.12600595  0.16634496  0.18049313  0.20986609
   0.23138332  0.26586011]
 [ 0.06266034  0.11125446  0.12600595  0.16634496  0.18049313  0.20986609
   0.23138332  0.26586011]
 [ 0.06266034  0.11125446  0.12600595  0.16634496  0.18049313  0.20986609
   0.23138332  0.26586011]
 [ 0.06266034  0.11125446  0.12600595  0.16634496  0.18049313  0.20986609
   0.23138332  0.26586011]
 [ 0.06266034  0.11125446  0.12600595  0.16634496  0.18049313  0.20986609
   0.23138332  0.26586011]]
